In [5]:
import pandas as pd
import os
import geopandas as gpd
import json
import requests

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

/home/ubuntu/real_estate_dashboard


# 데이터 불러 오기

In [116]:
# 인구데이터
census = pd.read_csv("0_data/census_202210.csv")

# 초등학교 데이터
elementary_school = pd.read_csv("0_data/elementary_school.csv")

# 전국 도시 공원 데이터
public_park = pd.read_csv("0_data/public_park.csv")

# 행정동코드 데이터
legal_info_h = pd.read_csv('0_data/legal_info_h.csv', 
                           dtype = {'행정동코드':object,
                                    '행정동시군구코드':object,
                                    '행정동읍면동코드':object})


# 행정동코드 데이터
legal_info_b = pd.read_csv('0_data/legal_info_b.csv', 
                           dtype = {'법정동코드':object,
                                    '행정동시군구코드':object,
                                    '행정동읍면동코드':object})

In [118]:
school_info = pd.read_csv('0_data/cleaning/school_cleaning.csv', dtype = {'법정동코드':object})
park_info = pd.read_csv('0_data/cleaning/park_lat_lon.csv')

# 인구 데이터 전처리

In [3]:
census.head(2).T

,0,1
TBL_NM,행정구역(읍면동)별/5세별 주민등록인구(2011년~),행정구역(읍면동)별/5세별 주민등록인구(2011년~)
PRD_DE,202210,202210
TBL_ID,DT_1B04005N,DT_1B04005N
ITM_NM,총인구수,남자인구수
ITM_NM_ENG,Population,Male
ITM_ID,T2,T3
UNIT_NM,명,명
ORG_ID,101,101
UNIT_NM_ENG,Person,Person
C1_OBJ_NM,행정구역(동읍면)별,행정구역(동읍면)별


In [4]:
census = census.astype({'PRD_DE':str,'C1':str})

census['year'] = census['PRD_DE'].str[:4]
census['month'] = census['PRD_DE'].str[4:]

census_2 = census[['year','month','ITM_NM', 'DT', 'C1','C1_NM']]

census_2.head()

,year,month,ITM_NM,DT,C1,C1_NM
0,2022,10,총인구수,51459626,0,전국
1,2022,10,남자인구수,25649509,0,전국
2,2022,10,여자인구수,25810117,0,전국
3,2022,10,총인구수,9443722,11,서울특별시
4,2022,10,남자인구수,4578459,11,서울특별시


In [5]:
census_2 = census_2[census_2['C1'].str.len() == 10] # 1, 2, 5, 10 
census_2 = census_2.reset_index(drop=True)

census_2.columns = ['년','월', '구분', '인구수', '행정동코드', '행정동']
census_2

,년,월,구분,인구수,행정동코드,행정동
0,2022,10,총인구수,11700,1111051500,청운효자동
1,2022,10,남자인구수,5367,1111051500,청운효자동
2,2022,10,여자인구수,6333,1111051500,청운효자동
3,2022,10,총인구수,9128,1111053000,사직동
4,2022,10,남자인구수,4045,1111053000,사직동
...,...,...,...,...,...,...
10726,2022,10,남자인구수,6222,5013061000,중문동
10727,2022,10,여자인구수,6037,5013061000,중문동
10728,2022,10,총인구수,3895,5013062000,예래동
10729,2022,10,남자인구수,1992,5013062000,예래동


In [6]:
census_3 = pd.merge(census_2, legal_info_h, 
        on = '행정동코드', 
        how = 'left')

In [7]:
census_3.head(2).T

,0,1
년,2022,2022
월,10,10
구분,총인구수,남자인구수
인구수,11700,5367
행정동코드,1111051500,1111051500
행정동,청운효자동,청운효자동
행정동_시도명,서울특별시,서울특별시
행정동_시군구명,종로구,종로구
행정동_읍면동명,청운효자동,청운효자동
행정동시군구코드,11110,11110


In [8]:
census_3.to_csv('0_data/cleaning/census_202210.csv', index = False)

# 초등학교 데이터 전처리

In [120]:
elementary_school_2 = pd.concat([elementary_school, school_info['법정동코드']], axis = 1)

In [122]:
elementary_school_2 = pd.merge(elementary_school_2, legal_info_b[['법정동코드', '시도명', '시군구명', '읍면동명']],
         on = '법정동코드',
         how = 'left'
        )

In [125]:
elementary_school_2.head(2)
elementary_school_2.to_csv('0_data/cleaning/elementary_school.csv', index = False)

# 전국 도시 공원 데이터 전처리

In [69]:
public_park['공원구분'] = public_park['공원구분'].str.replace('어린인공원', '어린이공원')
public_park_2 = pd.concat([public_park, park_info], axis = 1)


In [70]:
public_park_2.to_csv('0_data/cleaning/public_park.csv', index = False)